In [1]:
!conda install html5lib
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - html5lib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    html5lib-1.1               |             py_0          92 KB

The following packages will be UPDATED:

    html5lib: 1.0.1-py_0 --> 1.1-py_0


html5lib-1.1         | 92 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             

In [2]:
import numpy as np 
import pandas as pd
import json 
import requests 
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import lxml

#STEP_ONE: get data from the wiki url
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
# use BeautifulSoup to open it, define the referenced source list as canada_list
canada_list = BeautifulSoup(source, 'lxml') 
#creat the target dataframe columns and give them a name
toronto_list = pd.DataFrame(columns=['Postalcode','Borough', 'Neighborhood'])

#STEP_TWO: import loop process to get data from the web
#inspect structure: div(class)->table(class)->  {thead()->tr()->th(class)} OR {tbody()->tr()->td(table elements)}
content = canada_list.find('div', class_='mw-parser-output')
table = content.table.tbody 
postcode = 0
borough = 0
neighborhood = 0

#loop each elements into the table
for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:  #the first element followed by <td> as postcode
            postcode = td.text
            i = i + 1
        elif i == 1: #the second element followed by <td> as borough
            borough = td.text
            i = i + 1
        elif i == 2: #the third element followed by <td> as neighborhood
            neighborhood = td.text.strip('\n').replace(']','') #need to gather neighbors in one cell
    toronto_list = toronto_list.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

#STEP_THREE: Remove unwanted values
# remove Not assigned borough, both 'not assigned or = 0'
toronto_list = toronto_list[toronto_list.Borough!='Not assigned']
toronto_list = toronto_list[toronto_list.Borough!= 0]
toronto_list.reset_index(drop = True, inplace = True)

#
i = 0
for i in range(0,toronto_list.shape[0]):
    if toronto_list.iloc[i][2] == 'Not assigned':
        toronto_list.iloc[i][2] = toronto_list.iloc[i][1]
        i = i+1
        
#print the dataframe
df = toronto_list.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [3]:
df.shape


(180, 3)

In [4]:

#clean postalcode and meerge datasets
df_coord = pd.read_csv("http://cocl.us/Geospatial_data")
df_coord.rename(columns={'Postal Code':'Postalcode'}, inplace=True)
df_coord['Postalcode'] = df_coord['Postalcode'].astype(str)
df_coord['Postalcode'] = df_coord['Postalcode'].str.strip()
df['Postalcode'] = df['Postalcode'].astype(str)
df['Postalcode'] = df['Postalcode'].str.strip()
df1 = pd.merge(df, df_coord, how="inner", on = 'Postalcode')
df1.head(12)


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough\n,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough\n,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough\n,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough\n,Woburn,43.770992,-79.216917
4,M1H,Scarborough\n,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough\n,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough\n,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough\n,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough\n,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough\n,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans


In [6]:
# Step_1 examine the resulting dataframe.
neighborhoods = df1[['Borough','Neighborhood','Latitude','Longitude']]
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(neighborhoods['Borough'].unique()), neighborhoods.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


In [7]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer") #why it;s to_explorer?
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON are 43.6534817, -79.3839347.


In [9]:
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.4)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='#E42222',
        fill=True,
        fill_color='#CB9D5B',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [10]:
# Step_1 examine the resulting dataframe.
neighborhoods = df1[['Borough','Neighborhood','Latitude','Longitude']]
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(neighborhoods['Borough'].unique()), neighborhoods.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


In [11]:
# Step_2 Use geopy library to get the latitude and longitude values of Toronto.
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer") #why it;s to_explorer?
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON are 43.6534817, -79.3839347.


In [12]:
# Step_3 create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.4)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='#E42222',
        fill=True,
        fill_color='#CB9D5B',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
#Step_4 another object scaborough to analysis
scarborough_data = neighborhoods[neighborhoods['Borough'] == 'Scarborough'].reset_index(drop=True)
neighborhoods['Borough'] = neighborhoods['Borough'].str.strip()

scarborough_data.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476
